This notebook experiments with building out a fully customized TerriaJS catalog from a search result of ScienceBase Items, transforming sbJSON into the json catalog format used by TerriaJS. Resulting catalog JSON files are output to the local directory, deployed to GitHub, and then callable into an out of the box TerriaJS mapper.

In [20]:
import requests,json
from IPython.display import display

In [26]:
# Get what we want to work with using a ScienceBase /items/ query
sbURL = "https://www.sciencebase.gov/catalog/items?parentId=585400f6e4b0e2663625f36f&format=json&fields=title,body,distributionLinks&loadInstances=true"
sbJSON = requests.get(sbURL).json()
display (sbJSON)

{'items': [{'body': 'Spatial polygons of vegetation zones in 1986 for wetlands; P1, P2, P3, P4, P7, T3, T4, T5, T6, and T7 within the Cottonwood Lake Study Area, Stutsman County, North Dakota. Created from a collection of digital orthorectified images from aerial photographs of the study area acquired during 1986 using Stewart and Kantrud classification system.',
   'distributionLinks': [{'files': '',
     'name': '',
     'rel': 'alternate',
     'title': 'KML Service',
     'type': 'kml',
     'typeLabel': 'KML Download',
     'uri': 'https://www.sciencebase.gov/catalogMaps/mapping/ows/58c70390e4b0849ce978213f?mode=download&request=kml&service=wms&layers=Wetland_Vegetation_Zones_1986'},
    {'files': '',
     'name': '',
     'rel': 'alternate',
     'title': 'ScienceBase WMS Service',
     'type': 'serviceCapabilitiesUrl',
     'typeLabel': 'OGC Service Capabilities URL',
     'uri': 'https://www.sciencebase.gov/catalogMaps/mapping/ows/58c70390e4b0849ce978213f?service=wms&request=ge

In [28]:
theCatalog = {}
theCatalog["corsDomains"] = ["sciencebase.gov","corsproxy.com"]
theCatalog["baseMapName"] = "Bing Maps Roads"
theCatalog["initialViewerMode"] = "2d"
theCatalog["services"] = []

theCatalog["homeCamera"] = {}
theCatalog["homeCamera"]["west"] = -175
theCatalog["homeCamera"]["east"] = -66
theCatalog["homeCamera"]["north"] = 72
theCatalog["homeCamera"]["south"] = 18

theCatalog["catalog"] = []

padusGroup = {}
padusGroup["name"] = "Wetland Observatory Data"
padusGroup["type"] = "group"
padusGroup["isOpen"] = "true"

padusGroup["items"] = []

for idx, item in enumerate(sbJSON["items"]):
    thisItem = {}
    thisItem["id"] = item["id"]
    thisItem["name"] = item["title"]
    thisItem["description"] = item["body"]
    serviceCount = 0

#    if thisItem["name"] == "PADUS by Manager Name":
#        thisItem["layers"] = "0"
#        thisItem["isEnabled"] = "true"

    for link in item["distributionLinks"]:
        if link["title"] == "ScienceBase WMS Service":
            serviceCount = serviceCount + 1
            if serviceCount > 1:
                break
            thisItem["type"] = "wms"
            thisItem["url"] = link["uri"]
    padusGroup["items"].append(thisItem.copy())

theCatalog["catalog"].append(padusGroup.copy())

display (theCatalog)

f = open('catalog.json', 'wt', encoding='utf-8')
f.write(json.dumps(theCatalog).replace(': "true"', ': true'))

{'baseMapName': 'Bing Maps Roads',
 'catalog': [{'isOpen': 'true',
   'items': [{'description': 'Spatial polygons of vegetation zones in 1986 for wetlands; P1, P2, P3, P4, P7, T3, T4, T5, T6, and T7 within the Cottonwood Lake Study Area, Stutsman County, North Dakota. Created from a collection of digital orthorectified images from aerial photographs of the study area acquired during 1986 using Stewart and Kantrud classification system.',
     'id': '58c70390e4b0849ce978213f',
     'name': 'Cottonwood Lake Study Area-Wetland Vegetation Zones-1986',
     'type': 'wms',
     'url': 'https://www.sciencebase.gov/catalogMaps/mapping/ows/58c70390e4b0849ce978213f?service=wms&request=getcapabilities&version=1.3.0'},
    {'description': 'Spatial polygons of vegetation zones in 1990 for wetlands; P2, P3, P4, P6, P7, P8, P11, T2, T3, T4, T5, T6, T7, and T8 within the Cottonwood Lake Study Area, Stutsman County, North Dakota. Created from a collection of digital orthorectified images from aerial ph

12874